# Session 4 — Thinking Like an Agent

## Focus: Foundations of agent reasoning & tool use

In [ ]:
# Install required packages
!pip install langchain langchain-openai python-dotenv

# Key Concepts Deep Dive

### 1. ReAct (Reason + Act) Framework

The ReAct framework combines reasoning and acting in a loop:

1. **Reason**: The agent thinks about what it knows and what it needs to do next
2. **Act**: The agent takes action using available tools
3. **Observe**: The agent observes the results of its actions
4. **Repeat**: The cycle continues until the task is complete

This approach allows agents to handle complex tasks by breaking them down into smaller, manageable steps and using tools to gather information and perform actions.

### 2. Plan-and-Execute Architecture

The Plan-and-Execute architecture separates the planning and execution phases:

1. **Planning Phase**: The agent creates a detailed plan of action
2. **Execution Phase**: The agent executes the plan step by step
3. **Adjustment**: The agent can adjust the plan if needed based on execution results

This approach is particularly useful for complex, multi-step tasks that require careful planning and coordination of multiple tools.

### 3. Function Calling and Tool Design Patterns

Function calling allows LLMs to interact with external tools and APIs:

1. **Tool Definition**: Define functions that the agent can call
2. **Tool Description**: Provide clear descriptions of what each tool does
3. **Parameter Handling**: Define the parameters that each tool expects
4. **Result Processing**: Handle and process the results from tool calls

Common tool design patterns:
- **Information Retrieval**: Tools that fetch data from external sources
- **Data Processing**: Tools that process and transform data
- **Action Execution**: Tools that perform actions in the external world
- **Validation**: Tools that validate information or actions

### Hands-On: ReAct agent that plans a weekend trip

In [4]:
# First, let's set up our LLM
from langchain.agents import AgentType, initialize_agent, Tool
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
import re

# Initialize our LLM (using OpenRouter as before)
llm = ChatOpenAI(
    model="meta-llama/llama-3.1-8b-instruct",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key='sk-or-v1-6173516c9ecc07d1e10b606a570d1e73190a53974895d204a9fc40bb92a54c0fa', #remove a at the end to use the api key
    temperature=0,
    max_tokens=500,
)

# Create simulated APIs for our trip planning agent

# Simulated flight API
def flight_search(destination, date):
    """Simulate a flight search API"""
    flights = {
        "New York": {
            "2023-08-15": [
                {"airline": "Delta", "price": 350, "duration": "6h 15m"},
                {"airline": "United", "price": 320, "duration": "5h 45m"}
            ]
        },
        "Los Angeles": {
            "2023-08-15": [
                {"airline": "American", "price": 420, "duration": "5h 30m"},
                {"airline": "Southwest", "price": 380, "duration": "5h 45m"}
            ]
        },
        "Chicago": {
            "2023-08-15": [
                {"airline": "United", "price": 280, "duration": "2h 15m"},
                {"airline": "American", "price": 260, "duration": "2h 30m"}
            ]
        }
    }

    if destination in flights and date in flights[destination]:
        return flights[destination][date]
    else:
        return f"No flights found to {destination} on {date}"

# Simulated weather API
def get_weather(destination, date):
    """Simulate a weather API"""
    weather_data = {
        "New York": {
            "2023-08-15": {"temperature": 75, "condition": "Sunny", "humidity": 60}
        },
        "Los Angeles": {
            "2023-08-15": {"temperature": 85, "condition": "Clear", "humidity": 50}
        },
        "Chicago": {
            "2023-08-15": {"temperature": 70, "condition": "Partly Cloudy", "humidity": 65}
        }
    }

    if destination in weather_data and date in weather_data[destination]:
        return weather_data[destination][date]
    else:
        return f"Weather data not available for {destination} on {date}"

# Simulated hotel API
def find_hotels(destination, date, budget):
    """Simulate a hotel search API"""
    hotels = {
        "New York": {
            "2023-08-15": [
                {"name": "Grand Hotel", "price": 200, "rating": 4.5},
                {"name": "City Inn", "price": 150, "rating": 4.0}
            ]
        },
        "Los Angeles": {
            "2023-08-15": [
                {"name": "Sunset Resort", "price": 180, "rating": 4.3},
                {"name": "Beachside Hotel", "price": 160, "rating": 4.1}
            ]
        },
        "Chicago": {
            "2023-08-15": [
                {"name": "Downtown Suites", "price": 170, "rating": 4.2},
                {"name": "Riverfront Inn", "price": 140, "rating": 4.0}
            ]
        }
    }

    if destination in hotels and date in hotels[destination]:
        # Filter by budget
        available_hotels = [hotel for hotel in hotels[destination][date] if hotel["price"] <= budget]
        return available_hotels if available_hotels else f"No hotels found in {destination} within budget of ${budget}"
    else:
        return f"No hotels found in {destination} on {date}"

# Simulated attractions API
def find_attractions(destination):
    """Simulate an attractions API"""
    attractions = {
        "New York": [
            {"name": "Statue of Liberty", "type": "Landmark", "price": 25},
            {"name": "Central Park", "type": "Park", "price": 0},
            {"name": "Metropolitan Museum", "type": "Museum", "price": 30}
        ],
        "Los Angeles": [
            {"name": "Hollywood Walk of Fame", "type": "Landmark", "price": 0},
            {"name": "Santa Monica Pier", "type": "Entertainment", "price": 10},
            {"name": "Getty Center", "type": "Museum", "price": 0}
        ],
        "Chicago": [
            {"name": "Millennium Park", "type": "Park", "price": 0},
            {"name": "Art Institute", "type": "Museum", "price": 25},
            {"name": "Navy Pier", "type": "Entertainment", "price": 15}
        ]
    }

    if destination in attractions:
        return attractions[destination]
    else:
        return f"No attractions found for {destination}"

# Helper function to parse string inputs
def parse_input_string(input_str, expected_params):
    """Parse a string input into a dictionary of parameters"""
    params = {}
    # Simple parsing logic - this can be enhanced based on your needs
    for param in expected_params:
        # Look for patterns like "param: value" or "param=value"
        pattern = rf"{param}[:=]\s*([^,]+)"
        match = re.search(pattern, input_str, re.IGNORECASE)
        if match:
            params[param] = match.group(1).strip()

    return params

# Create wrapper functions that handle string inputs
def flight_search_wrapper(input_str):
    """Wrapper for flight search that handles string inputs"""
    params = parse_input_string(input_str, ["destination", "date"])
    return flight_search(params.get("destination", ""), params.get("date", ""))

def weather_search_wrapper(input_str):
    """Wrapper for weather search that handles string inputs"""
    params = parse_input_string(input_str, ["destination", "date"])
    return get_weather(params.get("destination", ""), params.get("date", ""))

def hotel_search_wrapper(input_str):
    """Wrapper for hotel search that handles string inputs"""
    params = parse_input_string(input_str, ["destination", "date", "budget"])
    # Convert budget to integer if possible
    try:
        budget = int(params.get("budget", "0"))
    except ValueError:
        budget = 0
    return find_hotels(params.get("destination", ""), params.get("date", ""), budget)

def attractions_search_wrapper(input_str):
    """Wrapper for attractions search that handles string inputs"""
    params = parse_input_string(input_str, ["destination"])
    return find_attractions(params.get("destination", ""))

# Create tools for our agent with the wrapper functions
tools = [
    Tool(
        name="FlightSearch",
        func=flight_search_wrapper,
        description="Search for flights to a destination on a specific date. Input should be in format: 'destination: Chicago, date: 2023-08-15'"
    ),
    Tool(
        name="GetWeather",
        func=weather_search_wrapper,
        description="Get weather information for a destination on a specific date. Input should be in format: 'destination: Chicago, date: 2023-08-15'"
    ),
    Tool(
        name="FindHotels",
        func=hotel_search_wrapper,
        description="Find hotels in a destination on a specific date within a budget. Input should be in format: 'destination: Chicago, date: 2023-08-15, budget: 200'"
    ),
    Tool(
        name="FindAttractions",
        func=attractions_search_wrapper,
        description="Find attractions in a destination. Input should be in format: 'destination: Chicago'"
    )
]

# Create a ReAct agent for trip planning
trip_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=6
)

# Test the trip planning agent
trip_query = "Plan a weekend trip to Chicago on August 15, 2023 with a budget of $500 for hotels"
print("Trip Planning Agent:")
print("=" * 50)
result = trip_agent.run(trip_query)
print(f"\nResult: {result}")

Trip Planning Agent:


> Entering new AgentExecutor chain...
Thought: I need to find flights to Chicago on August 15, 2023
Action: FlightSearch
Action Input: destination: Chicago, date: 2023-08-15
Observation: [{'airline': 'United', 'price': 280, 'duration': '2h 15m'}, {'airline': 'American', 'price': 260, 'duration': '2h 30m'}]
Thought:Thought: I need to find hotels in Chicago on August 15, 2023 within a budget of $500
Action: FindHotels
Action Input: destination: Chicago, date: 2023-08-15, budget: 500
Observation: [{'name': 'Downtown Suites', 'price': 170, 'rating': 4.2}, {'name': 'Riverfront Inn', 'price': 140, 'rating': 4.0}]
Thought:Question: Plan a weekend trip to Chicago on August 15, 2023 with a budget of $500 for hotels
Thought: I need to find flights to Chicago on August 15, 2023
Action: FlightSearch
Action Input: destination: Chicago, date: 2023-08-15
Observation: [{'airline': 'United', 'price': 280, 'duration': '2h 15m'}, {'airline': 'American', 'price': 260, 'duration': '2

### Case Study: Kayak-style concierge

**Introduction:**
In this case study, we'll create a Kayak-style concierge agent that helps users plan and book trips. This agent will combine multiple tools to provide a comprehensive travel planning experience.

**Key Requirements:**
1. Flight search and booking capabilities
2. Hotel search and booking capabilities
3. Weather information for destinations
4. Local attractions and activities
5. Budget planning and recommendations

**Architecture Design:**
Our concierge agent will use a ReAct architecture with the following components:

1. **Input Parser**: Extracts key information from user queries
2. **Tool Coordinator**: Manages the execution of various tools
3. **Response Generator**: Creates comprehensive responses based on tool outputs
4. **Booking System**: Handles the actual booking process (simulated)

**Implementation Approach:**
We'll create a specialized agent that can handle the complexity of travel planning by:
1. Breaking down user requests into specific tasks
2. Using appropriate tools for each task
3. Combining results into a cohesive plan
4. Providing recommendations based on the gathered information

In [5]:
# Create a specialized concierge agent
class TravelConciergeAgent:
    def __init__(self, llm, tools):
        self.llm = llm
        self.tools = tools
        self.agent = initialize_agent(
            tools,
            llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=8
        )

        # Create a prompt template for the concierge
        self.concierge_prompt = PromptTemplate(
            template="""You are a travel concierge assistant. Your goal is to help users plan and book trips.

            User request: {request}

            Please provide a comprehensive response that includes:
            1. Flight options (if requested)
            2. Hotel options (if requested)
            3. Weather information (if relevant)
            4. Local attractions and activities
            5. Any additional recommendations

            Response:""",
            input_variables=["request"]
        )

        self.concierge_chain = LLMChain(llm=llm, prompt=self.concierge_prompt)

    def plan_trip(self, request):
        """Plan a trip based on user request"""
        print(f"Planning trip for: {request}")
        print("=" * 60)

        # Use the agent to gather information
        try:
            # First, extract key information from the request
            info_extraction_prompt = f"""
            Extract the following information from the user request:
            - Destination
            - Dates
            - Budget
            - Any specific requirements or preferences

            Request: {request}

            Please respond with a JSON object containing the extracted information.
            """

            extracted_info = self.llm.invoke(info_extraction_prompt)
            print(f"Extracted information: {extracted_info.content}")

            # Use the agent to gather travel information
            travel_info = self.agent.run(request)
            print(f"\nTravel information gathered: {travel_info}")

            # Generate a comprehensive response
            response = self.concierge_chain.run(request=request)
            return response

        except Exception as e:
            return f"Error planning trip: {str(e)}"

# Initialize the concierge agent
concierge_agent = TravelConciergeAgent(llm, tools)

# Test the concierge agent
concierge_request = """
I want to plan a weekend trip to New York on August 15, 2023.
My budget for hotels is $200 per night.
I'm interested in museums and landmarks.
Please find flight options and hotel options for me.
"""

print("Travel Concierge Agent:")
print("=" * 50)
result = concierge_agent.plan_trip(concierge_request)
print(f"\nFinal Result: {result}")

C:\Users\Moonwalking\AppData\Local\Temp\ipykernel_4236\2876750199.py:32: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.concierge_chain = LLMChain(llm=llm, prompt=self.concierge_prompt)


Travel Concierge Agent:
Planning trip for: 
I want to plan a weekend trip to New York on August 15, 2023.
My budget for hotels is $200 per night.
I'm interested in museums and landmarks.
Please find flight options and hotel options for me.

Extracted information: Here is the extracted information in a JSON object:

```
{
  "destination": "New York",
  "dates": "August 15, 2023",
  "budget": {
    "hotel": 200
  },
  "preferences": {
    "interests": ["museums", "landmarks"],
    "requirements": []
  }
}
```

Note: I extracted the information as follows:

* Destination: "New York" (directly mentioned in the request)
* Dates: "August 15, 2023" (directly mentioned in the request)
* Budget: $200 per night for hotels (mentioned in the request)
* Preferences: I extracted the interests as "museums" and "landmarks" from the request, and left the requirements empty since there were no specific requirements mentioned.


> Entering new AgentExecutor chain...
Parsing LLM output produced both a fin

## Advanced Tool Design Patterns

In [7]:
# Advanced tool design patterns for agent development

# 1. Tool with validation
def validated_flight_search(destination, date):
    """Flight search with validation"""
    # Validate inputs
    if not isinstance(destination, str) or not destination:
        return "Error: Destination must be a non-empty string"

    if not isinstance(date, str) or not date:
        return "Error: Date must be a non-empty string"

    # Perform the search
    return flight_search(destination, date)

# 2. Tool with caching
flight_cache = {}

def cached_flight_search(destination, date):
    """Flight search with caching"""
    cache_key = f"{destination}_{date}"

    if cache_key in flight_cache:
        print(f"Using cached result for {cache_key}")
        return flight_cache[cache_key]

    result = flight_search(destination, date)
    flight_cache[cache_key] = result
    return result

# 3. Tool with fallback
def robust_flight_search(destination, date, max_retries=3):
    """Flight search with retry logic"""
    for attempt in range(max_retries):
        try:
            result = flight_search(destination, date)
            return result
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == max_retries - 1:
                return f"Error: Failed to search flights after {max_retries} attempts"

    return "Error: Unexpected error in flight search"

# 4. Composite tool
def comprehensive_travel_search(destination, date, budget):
    """Composite tool that searches flights, hotels, and weather"""
    flights = flight_search(destination, date)
    weather = get_weather(destination, date)
    hotels = find_hotels(destination, date, budget)
    attractions = find_attractions(destination)

    return {
        "flights": flights,
        "weather": weather,
        "hotels": hotels,
        "attractions": attractions
    }

# Create wrapper functions for advanced tools
def validated_flight_search_wrapper(input_str):
    """Wrapper for validated flight search"""
    params = parse_input_string(input_str, ["destination", "date"])
    return validated_flight_search(params.get("destination", ""), params.get("date", ""))

def cached_flight_search_wrapper(input_str):
    """Wrapper for cached flight search"""
    params = parse_input_string(input_str, ["destination", "date"])
    return cached_flight_search(params.get("destination", ""), params.get("date", ""))

def robust_flight_search_wrapper(input_str):
    """Wrapper for robust flight search"""
    params = parse_input_string(input_str, ["destination", "date"])
    return robust_flight_search(params.get("destination", ""), params.get("date", ""))

def comprehensive_travel_search_wrapper(input_str):
    """Wrapper for comprehensive travel search"""
    params = parse_input_string(input_str, ["destination", "date", "budget"])
    try:
        budget = int(params.get("budget", "0"))
    except ValueError:
        budget = 0
    return comprehensive_travel_search(params.get("destination", ""), params.get("date", ""), budget)

# Create tools with advanced patterns using the wrapper functions
advanced_tools = [
    Tool(
        name="ValidatedFlightSearch",
        func=validated_flight_search_wrapper,
        description="Search for flights with input validation. Input should be in format: 'destination: Chicago, date: 2023-08-15'"
    ),
    Tool(
        name="CachedFlightSearch",
        func=cached_flight_search_wrapper,
        description="Search for flights with caching. Input should be in format: 'destination: Chicago, date: 2023-08-15'"
    ),
    Tool(
        name="RobustFlightSearch",
        func=robust_flight_search_wrapper,
        description="Search for flights with retry logic. Input should be in format: 'destination: Chicago, date: 2023-08-15'"
    ),
    Tool(
        name="ComprehensiveTravelSearch",
        func=comprehensive_travel_search_wrapper,
        description="Comprehensive travel search including flights, hotels, weather, and attractions. Input should be in format: 'destination: Chicago, date: 2023-08-15, budget: 200'"
    )
]

# Create an agent with advanced tools
advanced_agent = initialize_agent(
    advanced_tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=6
)

# Test the advanced agent
advanced_query = "Use comprehensive travel search to plan a trip to Los Angeles on August 15, 2023 with a hotel budget of $200"
print("Advanced Travel Agent:")
print("=" * 50)
result = advanced_agent.run(advanced_query)
print(f"\nResult: {result}")

Advanced Travel Agent:


> Entering new AgentExecutor chain...
Thought: I need to search for flights, hotels, weather, and attractions for the trip
Action: ComprehensiveTravelSearch
Action Input: destination: Los Angeles, date: 2023-08-15, budget: 200
Observation: {'flights': [{'airline': 'American', 'price': 420, 'duration': '5h 30m'}, {'airline': 'Southwest', 'price': 380, 'duration': '5h 45m'}], 'weather': {'temperature': 85, 'condition': 'Clear', 'humidity': 50}, 'hotels': [{'name': 'Sunset Resort', 'price': 180, 'rating': 4.3}, {'name': 'Beachside Hotel', 'price': 160, 'rating': 4.1}], 'attractions': [{'name': 'Hollywood Walk of Fame', 'type': 'Landmark', 'price': 0}, {'name': 'Santa Monica Pier', 'type': 'Entertainment', 'price': 10}, {'name': 'Getty Center', 'type': 'Museum', 'price': 0}]}
Thought:Question: Use comprehensive travel search to plan a trip to Los Angeles on August 15, 2023 with a hotel budget of $200
Thought: I need to search for flights, hotels, weather, and attra

## Implementation Notes and Best Practices

In [9]:
# Best practices implementation for agent development
import time
from functools import lru_cache
from typing import Dict, List, Any
import re

class RobustTravelAgent:
    def __init__(self, llm, tools, max_iterations=5, timeout=30):
        self.llm = llm
        self.tools = tools
        self.max_iterations = max_iterations
        self.timeout = timeout
        self.execution_history = []

        # Create the agent with proper configuration
        self.agent = initialize_agent(
            tools,
            llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=self.max_iterations
        )

    def run_with_timeout(self, query):
        """Execute agent with timeout protection"""
        start_time = time.time()
        result = None
        error = None

        try:
            result = self.agent.run(query)
            self.execution_history.append({
                "query": query,
                "result": result,
                "success": True,
                "duration": time.time() - start_time
            })
        except Exception as e:
            error = str(e)
            self.execution_history.append({
                "query": query,
                "error": error,
                "success": False,
                "duration": time.time() - start_time
            })

            # Provide helpful error message
            if "iteration" in error.lower() or "time" in error.lower():
                result = "I need more time to process your request. Please try a simpler query or ask for specific information."
            else:
                result = f"I encountered an error while processing your request: {error}. Please try again with a different query."

        return result

    def get_recommendations(self, destination, budget=None):
        """Get personalized recommendations based on destination and budget"""
        # This could be enhanced with user preferences from a database
        recommendations = {
            "New York": [
                "Visit Central Park for a relaxing day outdoors",
                "See a Broadway show for entertainment",
                "Explore the Metropolitan Museum of Art"
            ],
            "Los Angeles": [
                "Visit Hollywood Walk of Fame",
                "Spend a day at Santa Monica Beach",
                "Tour the Getty Center museum"
            ],
            "Chicago": [
                "Visit Millennium Park and the Bean",
                "Explore the Art Institute of Chicago",
                "Take an architecture boat tour on the Chicago River"
            ]
        }

        base_recs = recommendations.get(destination, ["Explore local attractions and cuisine"])

        # Add budget-conscious recommendations
        if budget and budget < 150:
            budget_tips = [
                "Consider staying in budget-friendly neighborhoods",
                "Look for free museum days and attractions",
                "Use public transportation instead of taxis"
            ]
            return base_recs + budget_tips

        return base_recs

    def format_response(self, raw_result, query):
        """Format the agent response with clear structure and recommendations"""
        # Extract destination from query for personalized recommendations
        destination = None
        for city in ["New York", "Los Angeles", "Chicago"]:
            if city.lower() in query.lower():
                destination = city
                break

        # Extract budget information if available
        budget_match = re.search(r'\$(\d+)', query)
        budget = int(budget_match.group(1)) if budget_match else None

        # Create a structured response
        response = f"# Travel Planning Results\n\n"

        if "error" in str(raw_result).lower() or "no " in str(raw_result).lower():
            response += "I couldn't find complete information for your request. "
            response += "Here's what I can tell you:\n\n"
            response += f"**Raw result:** {raw_result}\n\n"
        else:
            response += "Here's the information I found:\n\n"
            response += f"{raw_result}\n\n"

        # Add personalized recommendations
        if destination:
            response += "## Personalized Recommendations\n\n"
            recs = self.get_recommendations(destination, budget)
            for i, rec in enumerate(recs, 1):
                response += f"{i}. {rec}\n"

            # Add general travel tips
            response += "\n## General Travel Tips\n\n"
            response += "1. Check weather forecasts before your trip\n"
            response += "2. Book accommodations in advance for better rates\n"
            response += "3. Consider travel insurance for unexpected events\n"
            response += "4. Check local COVID-19 guidelines and restrictions\n"

        return response

# Create a robust travel agent instance
robust_agent = RobustTravelAgent(
    llm=llm,
    tools=tools,
    max_iterations=4,  # Reduced to prevent infinite loops
    timeout=30
)

# Enhanced test function with best practices
def test_travel_scenarios(scenarios, agent):
    """Test the travel agent with multiple scenarios following best practices"""
    results = {}

    print("Testing Travel Agent Scenarios")
    print("=" * 60)

    for i, scenario in enumerate(scenarios, 1):
        print(f"\n{'='*50}")
        print(f"SCENARIO {i}: {scenario}")
        print(f"{'='*50}")

        try:
            # Execute with timeout protection
            start_time = time.time()
            raw_result = agent.run_with_timeout(scenario)
            execution_time = time.time() - start_time

            # Format the response
            formatted_result = agent.format_response(raw_result, scenario)

            results[scenario] = {
                "result": formatted_result,
                "error": None,
                "time": execution_time
            }

            print(f"Result: {formatted_result}")
            print(f"Execution time: {execution_time:.2f} seconds")

        except Exception as e:
            error_msg = f"Error: {str(e)}"
            results[scenario] = {
                "result": None,
                "error": error_msg,
                "time": 0
            }
            print(error_msg)

    # Print summary
    print("\n" + "=" * 60)
    print("TEST SUMMARY:")
    print("=" * 60)

    successful = sum(1 for r in results.values() if r["error"] is None)
    total_time = sum(r["time"] for r in results.values())

    print(f"Successful executions: {successful}/{len(scenarios)}")
    print(f"Total execution time: {total_time:.2f} seconds")
    print(f"Average time per query: {total_time/len(scenarios):.2f} seconds")

    return results

# Test with multiple travel scenarios
travel_scenarios = [
    "Find flights to Chicago on August 15, 2023",
    "Plan a weekend trip to Los Angeles with a hotel budget of $150 per night",
    "What's the weather like in New York on August 15, 2023?",
    "Find attractions in Chicago and recommend a hotel under $200"
]

# Run the tests
scenario_results = test_travel_scenarios(travel_scenarios, robust_agent)

# Display the guidelines
agent_development_guidelines()

# Additional monitoring and analytics
def display_agent_analytics(agent):
    """Display analytics about agent performance"""
    print("\n" + "=" * 60)
    print("AGENT PERFORMANCE ANALYTICS:")
    print("=" * 60)

    total_executions = len(agent.execution_history)
    successful_executions = sum(1 for e in agent.execution_history if e["success"])
    avg_duration = sum(e["duration"] for e in agent.execution_history) / total_executions if total_executions > 0 else 0

    print(f"Total executions: {total_executions}")
    print(f"Successful executions: {successful_executions}")
    print(f"Success rate: {(successful_executions/total_executions)*100:.1f}%" if total_executions > 0 else "N/A")
    print(f"Average execution time: {avg_duration:.2f} seconds")

    # Show recent executions
    print("\nRecent executions:")
    for i, execution in enumerate(agent.execution_history[-5:], 1):
        status = "✓" if execution["success"] else "✗"
        print(f"{i}. {status} {execution['query'][:50]}... ({execution['duration']:.2f}s)")

# Display analytics
display_agent_analytics(robust_agent)

Testing Travel Agent Scenarios

SCENARIO 1: Find flights to Chicago on August 15, 2023


> Entering new AgentExecutor chain...
Thought: I need to search for flights to a specific destination on a specific date.
Action: FlightSearch
Action Input: destination: Chicago, date: 2023-08-15
Observation: [{'airline': 'United', 'price': 280, 'duration': '2h 15m'}, {'airline': 'American', 'price': 260, 'duration': '2h 30m'}]
Thought:Question: Find flights to Chicago on August 15, 2023
Thought: I need to search for flights to a specific destination on a specific date.
Action: FlightSearch
Action Input: destination: Chicago, date: 2023-08-15
Observation: [{'airline': 'United', 'price': 280, 'duration': '2h 15m'}, {'airline': 'American', 'price': 260, 'duration': '2h 30m'}]
Thought:Question: Find flights to Chicago on August 15, 2023
Thought: I need to search for flights to a specific destination on a specific date.
Action: FlightSearch
Action Input: destination: Chicago, date: 2023-08-15
Observati